# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9998932d00>
├── 'a' --> tensor([[-0.1520, -1.2119, -1.1153],
│                   [ 0.8709, -0.5367, -1.1619]])
└── 'x' --> <FastTreeValue 0x7f9998932f70>
    └── 'c' --> tensor([[-0.8464, -0.9644,  0.2926, -0.5103],
                        [-0.4451,  0.9325, -0.9519,  0.1981],
                        [-0.7933, -2.3348,  0.0170, -0.4475]])

In [4]:
t.a

tensor([[-0.1520, -1.2119, -1.1153],
        [ 0.8709, -0.5367, -1.1619]])

In [5]:
%timeit t.a

69.1 ns ± 2.07 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9998932d00>
├── 'a' --> tensor([[-2.5590, -0.1931, -0.5310],
│                   [ 1.7426, -2.2369,  0.6950]])
└── 'x' --> <FastTreeValue 0x7f9998932f70>
    └── 'c' --> tensor([[-0.8464, -0.9644,  0.2926, -0.5103],
                        [-0.4451,  0.9325, -0.9519,  0.1981],
                        [-0.7933, -2.3348,  0.0170, -0.4475]])

In [7]:
%timeit t.a = new_value

73 ns ± 1.79 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1520, -1.2119, -1.1153],
               [ 0.8709, -0.5367, -1.1619]]),
    x: Batch(
           c: tensor([[-0.8464, -0.9644,  0.2926, -0.5103],
                      [-0.4451,  0.9325, -0.9519,  0.1981],
                      [-0.7933, -2.3348,  0.0170, -0.4475]]),
       ),
)

In [10]:
b.a

tensor([[-0.1520, -1.2119, -1.1153],
        [ 0.8709, -0.5367, -1.1619]])

In [11]:
%timeit b.a

65.5 ns ± 2.77 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4027,  0.0733,  1.2378],
               [-0.3370, -1.8475,  0.5882]]),
    x: Batch(
           c: tensor([[-0.8464, -0.9644,  0.2926, -0.5103],
                      [-0.4451,  0.9325, -0.9519,  0.1981],
                      [-0.7933, -2.3348,  0.0170, -0.4475]]),
       ),
)

In [13]:
%timeit b.a = new_value

578 ns ± 16.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.01 µs ± 23.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.2 µs ± 362 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

259 µs ± 8.78 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

266 µs ± 15.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f99989363a0>
├── 'a' --> tensor([[[-0.1520, -1.2119, -1.1153],
│                    [ 0.8709, -0.5367, -1.1619]],
│           
│                   [[-0.1520, -1.2119, -1.1153],
│                    [ 0.8709, -0.5367, -1.1619]],
│           
│                   [[-0.1520, -1.2119, -1.1153],
│                    [ 0.8709, -0.5367, -1.1619]],
│           
│                   [[-0.1520, -1.2119, -1.1153],
│                    [ 0.8709, -0.5367, -1.1619]],
│           
│                   [[-0.1520, -1.2119, -1.1153],
│                    [ 0.8709, -0.5367, -1.1619]],
│           
│                   [[-0.1520, -1.2119, -1.1153],
│                    [ 0.8709, -0.5367, -1.1619]],
│           
│                   [[-0.1520, -1.2119, -1.1153],
│                    [ 0.8709, -0.5367, -1.1619]],
│           
│                   [[-0.1520, -1.2119, -1.1153],
│                    [ 0.8709, -0.5367, -1.1619]]])
└── 'x' --> <FastTreeValue 0x7f99989364c0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

44.7 µs ± 611 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f98e952af70>
├── 'a' --> tensor([[-0.1520, -1.2119, -1.1153],
│                   [ 0.8709, -0.5367, -1.1619],
│                   [-0.1520, -1.2119, -1.1153],
│                   [ 0.8709, -0.5367, -1.1619],
│                   [-0.1520, -1.2119, -1.1153],
│                   [ 0.8709, -0.5367, -1.1619],
│                   [-0.1520, -1.2119, -1.1153],
│                   [ 0.8709, -0.5367, -1.1619],
│                   [-0.1520, -1.2119, -1.1153],
│                   [ 0.8709, -0.5367, -1.1619],
│                   [-0.1520, -1.2119, -1.1153],
│                   [ 0.8709, -0.5367, -1.1619],
│                   [-0.1520, -1.2119, -1.1153],
│                   [ 0.8709, -0.5367, -1.1619],
│                   [-0.1520, -1.2119, -1.1153],
│                   [ 0.8709, -0.5367, -1.1619]])
└── 'x' --> <FastTreeValue 0x7f98e9be43a0>
    └── 'c' --> tensor([[-0.8464, -0.9644,  0.2926, -0.5103],
                        [-0.4451,  0.9325, -0.9519,  0.1981],
                 

In [23]:
%timeit t_cat(trees)

39.9 µs ± 1.32 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

74.4 µs ± 1.21 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1520, -1.2119, -1.1153],
                [ 0.8709, -0.5367, -1.1619]],
       
               [[-0.1520, -1.2119, -1.1153],
                [ 0.8709, -0.5367, -1.1619]],
       
               [[-0.1520, -1.2119, -1.1153],
                [ 0.8709, -0.5367, -1.1619]],
       
               [[-0.1520, -1.2119, -1.1153],
                [ 0.8709, -0.5367, -1.1619]],
       
               [[-0.1520, -1.2119, -1.1153],
                [ 0.8709, -0.5367, -1.1619]],
       
               [[-0.1520, -1.2119, -1.1153],
                [ 0.8709, -0.5367, -1.1619]],
       
               [[-0.1520, -1.2119, -1.1153],
                [ 0.8709, -0.5367, -1.1619]],
       
               [[-0.1520, -1.2119, -1.1153],
                [ 0.8709, -0.5367, -1.1619]]]),
    x: Batch(
           c: tensor([[[-0.8464, -0.9644,  0.2926, -0.5103],
                       [-0.4451,  0.9325, -0.9519,  0.1981],
                       [-0.7933, -2.3348,  0.0170, -0.4475]],
         

In [26]:
%timeit Batch.stack(batches)

101 µs ± 2.18 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1520, -1.2119, -1.1153],
               [ 0.8709, -0.5367, -1.1619],
               [-0.1520, -1.2119, -1.1153],
               [ 0.8709, -0.5367, -1.1619],
               [-0.1520, -1.2119, -1.1153],
               [ 0.8709, -0.5367, -1.1619],
               [-0.1520, -1.2119, -1.1153],
               [ 0.8709, -0.5367, -1.1619],
               [-0.1520, -1.2119, -1.1153],
               [ 0.8709, -0.5367, -1.1619],
               [-0.1520, -1.2119, -1.1153],
               [ 0.8709, -0.5367, -1.1619],
               [-0.1520, -1.2119, -1.1153],
               [ 0.8709, -0.5367, -1.1619],
               [-0.1520, -1.2119, -1.1153],
               [ 0.8709, -0.5367, -1.1619]]),
    x: Batch(
           c: tensor([[-0.8464, -0.9644,  0.2926, -0.5103],
                      [-0.4451,  0.9325, -0.9519,  0.1981],
                      [-0.7933, -2.3348,  0.0170, -0.4475],
                      [-0.8464, -0.9644,  0.2926, -0.5103],
                      [-0.4451,  

In [28]:
%timeit Batch.cat(batches)

174 µs ± 2.97 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

518 µs ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
